In [1]:
import torch
import syft

In [2]:
hook = syft.TorchHook(torch)

In [3]:
alice = syft.VirtualWorker(hook,id="alice")
bob = syft.VirtualWorker(hook,id="bob")

In [4]:
Q_BITS = 7
field = 2**Q_BITS -1
def decompose(shares):
   """decompose a tensor into its binary representation."""
   powers = torch.arange(Q_BITS)
   locations = list(shares.keys())
   shares_shape = shares[locations[0]].shape

   for i in range(len(shares_shape)):
       powers = powers.unsqueeze(0)
   moduli = 2 ** powers

   decomposition = {}
   for location in locations:
       share = shares[location].unsqueeze(-1)
       moduli_ptr = moduli.send(location)

       decomposition[location] = torch.remainder(
           ((share + 2 ** (Q_BITS)) / moduli_ptr.type_as(share))
           , 2
       )
   return decomposition

In [5]:
t = torch.tensor([1, 2, 3, 4.0])
x = t.fix_prec(field=field).share(bob, alice,field=field)

In [6]:
x

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:35115250218 -> bob:78790786047]
	-> (Wrapper)>[PointerTensor | me:904814793 -> alice:1357788618]

In [22]:
print(alice._objects)

{1357788618: tensor([-133,  -75,  -89,  -52])}


In [21]:
print(bob._objects)

{78790786047: tensor([117,  43,  41, 115])}


In [7]:
decomposed = decompose(x.child.child.child.child)

In [8]:
decomposed

{'bob': (Wrapper)>[PointerTensor | me:32723485222 -> bob:32723485222],
 'alice': (Wrapper)>[PointerTensor | me:97169878507 -> alice:97169878507]}

In [9]:
a= decomposed["alice"].get()

In [10]:
a

tensor([[1, 0, 1, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 1, 0],
        [1, 1, 1, 0, 0, 1, 0],
        [0, 0, 1, 1, 0, 0, 1]])

In [11]:
b= decomposed["bob"].get()

In [12]:
b

tensor([[1, 0, 1, 0, 1, 1, 1],
        [1, 1, 0, 1, 0, 1, 0],
        [1, 0, 0, 1, 0, 1, 0],
        [1, 1, 0, 0, 1, 1, 1]])

In [13]:
def convert_to_int(bits):
    total = 0
    for i, bit in enumerate(bits):
        total += bit * 2**i
    return total

In [18]:
convert_to_int([1, 0, 1, 0, 0, 0, 0])

5

In [19]:
convert_to_int([1, 0, 1, 0, 1, 1, 1])

117

In [20]:
field

127

In [26]:
class test_base():
    def __init__(self):
        print(type(self))
class not_base(test_base):
    def __init__(self):
        print("testing")
        super().__init__()

test_base()

<class '__main__.test_base'>
